## 2. Марковский процесс принятия решения (markov decison process)

### Задание
#### 1. Приведите жизненный пример марковского процесса принятия решения (это может быть какаю-нибудь игра и т.п.).
Очень простой пример - шахматы. Конечное, пусть и очень большое, количество состояний $S$, конечное множество действий $ A \subset S \times S$. Среда - наш противник - отвечает вполне себе случайно. Если рассматривать среду именно как противника, то она, конечно, отдаёт сильное предпочтение тем действиям, которые она считает выигрышными. Награда для каждого допустимого действия из $\; S \times A \;$ нулевая за исключением последнего хода, по результатам которого присуждается награда $\pm 1$. Кроме того, очевидно, что для принятия решения информация о предыдущих состояниях нерелевантна. Таким образом, принятие решения в шахматах можно смоделировать с помощью марковского процесса.
#### 2. Можете ли вы привести пример игры, где принятие решения нельзя смоделировать с помощью марковского процесса?
Крестики-нолики на бесконечной доске не могут быть смоделированы хотя бы из-за бесконечности (в частности, континуальности) множества состояний $S$.
#### 3. Выведите следующие значения через $p(s_{t+1}, r_{t+1}|s_t, a_t)$, для простоты все распределения можно считать дискретными
  * $r(s_{t}, a_{t}) = \mathbb{E}[R_{t+1}|S_t = s_t, A_t = a_t]$ - средняя награда за действие $a_t$ в $s_t$ 
  * $p(s_{t+1} | s_t, a_t) = \Pr\{S_{t+1} = s_{t+1} | S_t = s_t, A_t = a_t \}$ - вероятность попасть в $s_{t+1}$ из $s_t$, сделав $a_t$.
  * $r(s_t, a_t, s_{t+1}) = \mathbb{E}[R_{t+1}|S_{t+1} = s_{t+1}, S_t = s_t, A_t = a_t]$ - средняя награда при переезде из $s_t$ в $s_{t+1}$, сделав $a_t$.

$r(s_{t}, a_{t}) = \mathbb{E}[R_{t+1}|S_t = s_t, A_t = a_t] = \sum_{s,r} r \cdot p(s, r \;|\; s_t, a_t)$

$p(s_{t+1} | s_t, a_t) = \Pr\{S_{t+1} = s_{t+1} | S_t = s_t, A_t = a_t \} = \sum_{r} p(s_{t+1}, r \;|\; s_t, a_t)$

$r(s_t, a_t, s_{t+1}) = \mathbb{E}[R_{t+1}|S_{t+1} = s_{t+1}, S_t = s_t, A_t = a_t] = \sum_{r} r \cdot p(s_{t+1}, r \;|\; s_t, a_t)$



### Смоделируем среду:

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.ndimage.filters import gaussian_filter1d


def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [28]:
class Environment:
    def __init__(self, states, actions):
        self.states = states
        self.actions = actions
        self.reward_means = np.random.normal(0, 1, size=(states, actions))
        transitions = np.random.randint(0, 2, size=(states, actions, states), dtype='int8')
        transitions[:, :, 0] += 1 - transitions.max(2)
        self.transition_probas = np.random.random_sample((states, actions, states)).astype('float64') * transitions
        for state in self.transition_probas:
            for action in state:
                if np.absolute(action).sum() < 0.01:
                    action = np.ones(states)
                action /= action.sum()
    
    def step(self, state, action):
        return (np.random.choice(self.states, p=self.transition_probas[state, action]),
            np.random.normal(self.reward_means[state, action], 1))

In [39]:
class PolicyIterationStrategy:
    def __init__(self, env, discount):
        self.env = env
        self.discount = discount
        self.policy = np.ones((env.states, env.actions))
        self.policy /= env.actions
    def learn(self, steps):
        state_values = np.random.randn(self.env.states)
        state_action_values = np.zeros((self.env.states, self.env.actions))
        i = 0
        while i < steps: # TODO: decide when to stop
            for state in range(self.env.states):
                val_sum = 0
                for action in range(self.env.actions):
                    action_sum = 0
                    for new_state in range(self.env.states):
                        action_sum += self.env.transition_probas[state, action, new_state] * \
                            (self.env.reward_means[state, action] + \
                                self.discount*state_values[new_state])
                    val_sum += self.policy[state, action] * action_sum
                    state_action_values[state, action] = action_sum
                state_values[state] = val_sum
                comp_values = state_action_values[state]
                best = np.argwhere(comp_values == comp_values.max())
                self.policy[state] = 0
                self.policy[state][best] = 1
                self.policy[state] /= self.policy[state].sum()
                i += 1
                if i >= steps:
                    break
                
    def choose(self, state):
        return np.random.choice(self.env.actions, p=self.policy[state])
    

class ValueIterationStrategy:
    def __init__(self, env, discount):
        self.env = env
        self.discount = discount
        self.policy = np.ones((env.states, env.actions))
        self.policy /= env.actions
    def learn(self, steps):
        state_values = np.random.randn(self.env.states)
        state_action_values = np.zeros((self.env.states, self.env.actions))
        i = 0
        while i < steps: # TODO: decide when to stop
            for state in range(self.env.states):
                max_val = -1e20
                for action in range(self.env.actions):
                    action_sum = 0
                    for new_state in range(self.env.states):
                        action_sum += self.env.transition_probas[state, action, new_state] * \
                            (self.env.reward_means[state, action] + \
                                self.discount*state_values[new_state])
                    max_val = max(max_val, action_sum)
                    state_action_values[state, action] = action_sum
                state_values[state] = max_val
                comp_values = state_action_values[state]
                best = np.argwhere(comp_values == comp_values.max())
                self.policy[state] = 0
                self.policy[state][best] = 1
                self.policy[state] /= self.policy[state].sum()
                i += 1
                if i >= steps:
                    break
                
    def choose(self, state):
        return np.random.choice(self.env.actions, p=self.policy[state])
    
class RandomStrategy:
    def __init__(self, env, discount):
        self.env = env
        self.discount = discount
        self.policy = np.ones((env.states, env.actions))
        self.policy /= env.actions
    def learn(self, steps):
        pass
    def choose(self, state):
        return np.random.choice(self.env.actions, p=self.policy[state])

In [43]:
class MarkovPlayer:
    def __init__(self, states, actions, steps, strategy_class, **kwargs):
        self.states = states
        self.actions = actions
        self.strategy_class = strategy_class
        self.steps = steps
        self.strategy_args = kwargs
    def evaluate(self, games=1000, learning_time=100, progressbar=True,hold=False, show=True,
                 color='b', label=' '):
        rewards = np.zeros(self.steps)
#         env = global_env
        if progressbar:
            games_range = log_progress(range(games), every=1)
        else:
            games_range = range(games)
        for game in games_range:
            state = 0
            self.state = state
            env = Environment(self.states, self.actions)
            strategy = self.strategy_class(env, **(self.strategy_args))
            self.strategy = strategy
            strategy.learn(learning_time)
            for i in range(self.steps):
                state, rewards[i] = env.step(state, strategy.choose(state))
                self.state = state
        for i in range(1, self.steps):
            rewards[i] += rewards[i-1]
        if show:
            x = np.arange(1, self.steps+1)
            plot_rewards = rewards
            plt.plot(x, plot_rewards, color, label=label)
            plt.title('Total reward', fontsize=16)
            if not hold:
                plt.show()
        return rewards.sum()

In [45]:
games_num = 200
random_player = MarkovPlayer(states=100, actions=20, steps=1000,
                           strategy_class=RandomStrategy, discount=1.0)
policy_iteration_player = MarkovPlayer(states=100, actions=20, steps=1000,
                           strategy_class=PolicyIterationStrategy, discount=1.0)
value_iteration_player = MarkovPlayer(states=100, actions=20, steps=1000,
                           strategy_class=ValueIterationStrategy, discount=1.0)
# global_env = Environment(100, 20)
plt.figure(figsize=(10, 6))
plt.subplot(121)
plt.hold(True)
random_player.evaluate(games=games_num, hold=True, color='k')
policy_iteration_player.evaluate(games=games_num, learning_time=1, hold=True, color='r')
policy_iteration_player.evaluate(games=games_num, learning_time=10, hold=True, color='g')
policy_iteration_player.evaluate(games=games_num, learning_time=25, hold=True, color='b')
policy_iteration_player.evaluate(games=games_num, learning_time=100, hold=True, color='m')
plt.hold(False)
plt.axis([0, 1000, 0, 2000])
plt.subplot(122)
plt.hold(True)
random_player.evaluate(games=games_num, hold=True, color='k')
value_iteration_player.evaluate(games=games_num, learning_time=1, hold=True, color='r')
value_iteration_player.evaluate(games=games_num, learning_time=10, hold=True, color='g')
value_iteration_player.evaluate(games=games_num, learning_time=25, hold=True, color='b')
value_iteration_player.evaluate(games=games_num, learning_time=100, hold=True, color='m')
plt.hold(False)
plt.axis([0, 1000, 0, 2000])
plt.show()

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:11: MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
/usr/local/lib/python3.5/dist-packages/matplotlib/__init__.py:917: UserWarning: axes.hold is deprecated. Please remove it from your matplotlibrc and/or style files.
  warnings.warn(self.msg_depr_set % key)
/usr/local/lib/python3.5/dist-packages/matplotlib/rcsetup.py:152: UserWarning: axes.hold is deprecated, will be removed in 3.0
  warnings.warn("axes.hold is deprecated, will be removed in 3.0")


KeyboardInterrupt: 

In [ ]:
# print(policy_iteration_player.env.state, policy_iteration_player.env.action)
# print(policy_iteration_player.env.transition_probas[0, 0])
print(policy_iteration_player.strategy.policy)

In [ ]:
# ATTENTION: takes a significant amount of time to be processed
games_num = 5
    
# learning_steps = [1, 2, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
learning_steps = [1, 2, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31]
policy_results = []
for e in log_progress(learning_steps, every=1):
    player = MarkovPlayer(states=100, actions=20, steps=1000,
                           strategy_class=PolicyIterationStrategy, discount=1.0)
    res = player.evaluate(games=games_num, learning_time=e, show=False, progressbar=False)
    policy_results.append(res)
    
value_results = []
for e in log_progress(learning_steps, every=1):
    player = MarkovPlayer(states=100, actions=20, steps=1000,
                           strategy_class=ValueIterationStrategy, discount=1.0)
    res = player.evaluate(games=games_num, learning_time=e, show=False, progressbar=False)
    value_results.append(res)

plt.figure(figsize=(8, 5))
plt.title('score', fontsize=16)
plt.axis([0, learning_steps[-1],
          0, max(*policy_results, *value_results)*1.1])
plt.xlabel('learning games', fontsize=12)
plt.ylabel('total reward', fontsize=12)
plt.hold(True)
plt.plot(learning_steps, policy_results, 'r', label='Policy', linewidth=1.5)
plt.plot(learning_steps, value_results, 'b', label='Value', linewidth=1.5)
plt.hold(False)
plt.legend(loc='lower center', fontsize=14, ncol=2)
plt.show()